# Refinamento dos dados de viagens de táxis de Nova York - dados da NYC

* Base de dados com colunas selecionadas e prontas para consumo analítico
* Padronização das coluans entre os veículos
* Unificação das tabelas dos táxis amarelos e verdes
* Consistência dos dados

## Camada gold

## Bibliotecas

In [0]:
from pyspark.sql.functions import col, year, month, dayofmonth, hour, lit, coalesce
from datetime import datetime
from dateutil.rrule import rrule, MONTHLY

## Parâmetros

In [0]:
# Configuração Inicial
dbutils.widgets.text("start_date", "2023-01", "Data Início (YYYY-MM)")
dbutils.widgets.text("end_date", "2023-01", "Data Fim (YYYY-MM)")

In [0]:
# Obter Parâmetros
start_date = dbutils.widgets.get("start_date")
end_date = dbutils.widgets.get("end_date")

print(f"Processando camada Gold unificada (yellow/green) de {start_date} a {end_date}")

Processando camada Gold unificada (yellow/green) de 2023-01 a 2023-05


## Caminhos

In [0]:
# Configuração de Caminhos
silver_base_path = "/case_ifood_nyc/silver/nyc_taxi_delta"
gold_path = "/case_ifood_nyc/gold/nyc_taxi_unified"
dbutils.fs.mkdirs(gold_path)

Out[21]: True

## Funções

In [0]:
# Função para processar cada partição
def process_gold_partition(vehicle_type, year_month):
    try:
        year_part, month_part = year_month.split("-")
        print(f"\nProcessando {vehicle_type} - {year_month}...")
        
        # 1. Ler dados da Silver
        df = spark.read.format("delta").load(f"{silver_base_path}/{vehicle_type}/year={year_part}/month={month_part}")
        
        # 2. Tratar colunas específicas de cada tipo
        if vehicle_type == "yellow":
            pickup_col = "tpep_pickup_datetime"
            dropoff_col = "tpep_dropoff_datetime"
        else:  # green
            pickup_col = "lpep_pickup_datetime"
            dropoff_col = "lpep_dropoff_datetime"
        
        # 3. Selecionar e transformar colunas
        df_gold = df.select(
            lit(vehicle_type).alias("vehicle_type"),
            coalesce(col("VendorID"), col("vendorid")).cast("int").alias("VendorID"),
            col("passenger_count").cast("int").alias("passenger_count"),
            col("total_amount").cast("double").alias("total_amount"),
            col(pickup_col).cast("timestamp").alias("pickup_datetime"),
            col(dropoff_col).cast("timestamp").alias("dropoff_datetime"),
            lit(int(year_part)).alias("meta_file_year"),
            lit(int(month_part)).alias("meta_file_month")
        )
        
        # 4. Adicionar colunas derivadas
        df_gold = (df_gold
            .withColumn("pickup_year", year(col("pickup_datetime")))
            .withColumn("pickup_month", month(col("pickup_datetime")))
            .withColumn("pickup_day", dayofmonth(col("pickup_datetime")))
            .withColumn("pickup_hour", hour(col("pickup_datetime")))
            .withColumn("trip_duration_minutes", 
                       (col("dropoff_datetime").cast("long") - 
                        col("pickup_datetime").cast("long")) / 60)
            .filter(col("pickup_datetime").isNotNull() & 
                   col("dropoff_datetime").isNotNull()))
        
        return df_gold
        
    except Exception as e:
        print(f"❌ Erro ao processar {vehicle_type} - {year_month}: {str(e)}")
        return None

## Tratamento

In [0]:
# Schema esperado para a tabela unificada
unified_schema = """
    vehicle_type STRING,
    VendorID INT,
    passenger_count INT,
    total_amount DOUBLE,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    pickup_year INT,
    pickup_month INT,
    pickup_day INT,
    pickup_hour INT,
    trip_duration_minutes DOUBLE,
    meta_file_year INT,
    meta_file_month INT
"""

In [0]:
start = datetime.strptime(start_date, "%Y-%m")
end = datetime.strptime(end_date, "%Y-%m")
vehicle_types = ["yellow", "green"]  # Apenas yellow e green
success_count = 0

# Limpar dados anteriores
dbutils.fs.rm(gold_path, recurse=True)

Out[24]: True

In [0]:
# Lista dos DataFrames processados
all_dfs = []

for vehicle_type in vehicle_types:
    for dt in rrule(MONTHLY, dtstart=start, until=end):
        year_month = dt.strftime("%Y-%m")
        df_processed = process_gold_partition(vehicle_type, year_month)
        if df_processed is not None:
            all_dfs.append(df_processed)
            success_count += 1


Processando yellow - 2023-01...

Processando yellow - 2023-02...

Processando yellow - 2023-03...

Processando yellow - 2023-04...

Processando yellow - 2023-05...

Processando green - 2023-01...

Processando green - 2023-02...

Processando green - 2023-03...

Processando green - 2023-04...

Processando green - 2023-05...


In [0]:
# Unir todos os DataFrames e salvar
if all_dfs:
    # Combinar todos os DataFrames
    gold_df = all_dfs[0]
    for df in all_dfs[1:]:
        gold_df = gold_df.union(df)
    
    # Salvar na camada Gold
    (gold_df.write
       .format("delta")
       .mode("overwrite")
       .partitionBy("vehicle_type", "meta_file_year", "meta_file_month")
       .save(gold_path))
    
    print(f"\n✅ Salvando dados Gold unificados em: {gold_path}")


✅ Salvando dados Gold unificados em: /case_ifood_nyc/gold/nyc_taxi_unified


In [0]:
# Registrar Tabela Gold Unificada
if success_count > 0:
    print("\nRegistrando tabela Gold unificada...")
    spark.sql("CREATE DATABASE IF NOT EXISTS case_ifood_nyc_taxi")
    
    # Criar tabela com schema explícito
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS case_ifood_nyc_taxi.gold_unified_taxi (
        {unified_schema}
    )
    USING DELTA
    PARTITIONED BY (vehicle_type, meta_file_year, meta_file_month)
    LOCATION '{gold_path}'
    """)
    
    # Otimizar tabela
    spark.sql(f"OPTIMIZE case_ifood_nyc_taxi.gold_unified_taxi")
    print("\n✅ Tabela Gold unificada criada com sucesso!")
    
    # Verificação
    print("\n📊 Amostra de dados:")
    display(spark.table("case_ifood_nyc_taxi.gold_unified_taxi").limit(5))
    
else:
    print("\n❌ Nenhum dado foi processado com sucesso na camada Gold.")


Registrando tabela Gold unificada...

✅ Tabela Gold unificada criada com sucesso!

📊 Amostra de dados:


vehicle_type,VendorID,passenger_count,total_amount,pickup_datetime,dropoff_datetime,meta_file_year,meta_file_month,pickup_year,pickup_month,pickup_day,pickup_hour,trip_duration_minutes
yellow,2,1,14.3,2023-01-01T00:32:10.000+0000,2023-01-01T00:40:36.000+0000,2023,1,2023,1,1,0,8.433333333333334
yellow,2,1,16.9,2023-01-01T00:55:08.000+0000,2023-01-01T01:01:27.000+0000,2023,1,2023,1,1,0,6.316666666666666
yellow,2,1,34.9,2023-01-01T00:25:04.000+0000,2023-01-01T00:37:49.000+0000,2023,1,2023,1,1,0,12.75
yellow,1,1,20.85,2023-01-01T00:03:48.000+0000,2023-01-01T00:13:25.000+0000,2023,1,2023,1,1,0,9.616666666666667
yellow,2,1,19.68,2023-01-01T00:10:29.000+0000,2023-01-01T00:21:19.000+0000,2023,1,2023,1,1,0,10.833333333333334
